# Deep Learning Toolkit for Splunk - Notebook for TensorFlow 2.0

## Autoencoder Example
This notebook contains an example workflow how to work on custom containerized code that seamlessly interfaces with the Deep Learning Toolkit for Splunk. As an example we use a custom autoencoder built on keras and tensorflow.

Note: By default every time you save this notebook the cells are exported into a python module which is then invoked by Splunk MLTK commands like <code> | fit ... | apply ... | summary </code>. Please read the Model Development Guide in the Deep Learning Toolkit app for more information.

## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [ ]:
# mltkc_import
# this definition exposes all python module imports that should be available in all subsequent commands
import json
import datetime
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

# global constants
MODEL_DIRECTORY = "/srv/app/model/data/"

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing purposes
print("numpy version: " + np.__version__)
print("pandas version: " + pd.__version__)
print("TensorFlow version: " + tf.__version__)
print("Keras version: " + keras.__version__)

## Stage 1 - get a data sample from Splunk
In Splunk run a search to pipe a prepared dataset into this environment.

| inputlookup iris.csv <br>| fit MLTKContainer algo=autoencoder epochs=100 batch_size=4 components=2 petal_length petal_width sepal_length sepal_width into app:iris_autoencoder

After you run this search your data set sample is available as a csv inside the container to develop your model. The name is taken from the into keyword ("my_model" in the example above) or set to "default" if no into keyword is present. This step is intended to work with a subset of your data to create your custom model.

In [ ]:
# mltkc_stage
# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
def stage(name):
    with open("data/"+name+".csv", 'r') as f:
        df = pd.read_csv(f)
    with open("data/"+name+".json", 'r') as f:
        param = json.load(f)
    return df, param

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing purposes
df, param = stage("iris_autoencoder")
print(df[0:1])
print(df.shape)
print(str(param))

## Stage 2 - create and initialize a model

In [ ]:
# mltkc_init
# initialize the model
# params: data and parameters
# returns the model object which will be used as a reference to call fit, apply and summary subsequently
def init(df,param):
    X = df[param['feature_variables']]
    print("FIT build model with input shape " + str(X.shape))
    components = 3
    activation_fn = 'relu'
    # learning_rate = 0.001
    # epsilon=0.00001 # default 1e-07
    if 'options' in param:
        if 'params' in param['options']:
            if 'components' in param['options']['params']:
                components = int(param['options']['params']['components'])
            if 'activation_func' in param['options']['params']:
                activation_fn = param['options']['params']['activation_func']
    input_shape = int(X.shape[1])
    encoder_layer = keras.layers.Dense(components, input_dim=input_shape, activation=activation_fn, kernel_initializer=keras.initializers.RandomUniform(minval=0, maxval=1, seed=None), bias_initializer=keras.initializers.RandomUniform(minval=0, maxval=1, seed=None))
    decoder_layer = keras.layers.Dense(input_shape, activation=activation_fn, kernel_initializer=keras.initializers.RandomUniform(minval=0, maxval=1, seed=None), bias_initializer=keras.initializers.RandomUniform(minval=0, maxval=1, seed=None))
    model = keras.Sequential()
    model.add(encoder_layer)
    model.add(decoder_layer)
    #opt = keras.optimizers.Adam(learning_rate=learning_rate, epsilon=epsilon)
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
# test mltkc_stage_create_model
model = init(df,param)
print(model.summary())

## Stage 3 - fit the model

In [ ]:
# mltkc_stage_create_model_fit
# returns a fit info json object
def fit(model,df,param):
    returns = {}
    X = df[param['feature_variables']]
    model_epochs = 100
    model_batch_size = 32
    if 'options' in param:
        if 'params' in param['options']:
            if 'epochs' in param['options']['params']:
                model_epochs = int(param['options']['params']['epochs'])
            if 'batch_size' in param['options']['params']:
                model_batch_size = int(param['options']['params']['batch_size'])
    # connect model training to tensorboard
    log_dir="/srv/notebooks/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    # run the training
    returns['fit_history'] = model.fit(x=X,
                                       y=X, 
                                       verbose=2, 
                                       epochs=model_epochs, 
                                       batch_size=model_batch_size, 
                                       #validation_data=(X, Y),
                                       callbacks=[tensorboard_callback])
    # memorize parameters
    returns['model_epochs'] = model_epochs
    returns['model_batch_size'] = model_batch_size
    returns['model_loss_acc'] = model.evaluate(x = X, y = X)
    return returns

In [ ]:
returns = fit(model,df,param)
print(returns['model_loss_acc'])

## Stage 4 - apply the model

In [ ]:
# mltkc_stage_create_model_apply
def apply(model,df,param):
    X = df[param['feature_variables']]
    reconstruction = model.predict(x = X)
    intermediate_layer_model = keras.Model(inputs=model.input, outputs=model.layers[0].output)
    hidden = intermediate_layer_model.predict(x = X)
    y_hat = pd.concat([pd.DataFrame(reconstruction).add_prefix("reconstruction_"), pd.DataFrame(hidden).add_prefix("hidden_")], axis=1)
    return y_hat

In [ ]:
# test mltkc_stage_create_model_apply
y_hat = apply(model,df,param)
print(y_hat)

## Stage 5 - save the model

In [ ]:
# save model to name in expected convention "<algo_name>_<model_name>.h5"
def save(model,name):
    # save keras model to hdf5 file
    # https://www.tensorflow.org/beta/tutorials/keras/save_and_restore_models
    model.save(MODEL_DIRECTORY + name + ".h5")
    return model

## Stage 6 - load the model

In [ ]:
# load model from name in expected convention "<algo_name>_<model_name>.h5"
def load(name):
    model = keras.models.load_model(MODEL_DIRECTORY + name + ".h5")
    return model

## Stage 7 - provide a summary of the model

In [ ]:
# return model summary
def summary(model=None):
    returns = {"version": {"tensorflow": tf.__version__, "keras": keras.__version__} }
    if model is not None:
        # Save keras model summary to string:
        s = []
        model.summary(print_fn=lambda x: s.append(x+'\n'))
        returns["summary"] = ''.join(s)
    return returns

## End of Stages
All subsequent cells are not tagged and can be used for further freeform code